In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

C:\Users\Ryan's PC\AppData\Local\Temp\ipykernel_23984\2895603275.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Data Cleaning (KaggleMovies.csv)

In [2]:
df = pd.read_csv('Datasets/KaggleMovies.csv')
pd.set_option('float_format', '{:f}'.format)

In [3]:
# Renaming columns
column_mapping = {
    'name' : 'Name',
    'rating' : 'Rating',
    'genre' : 'Genre',
    'year' : 'Year',
    'released' : 'Released',
    'score' : 'Score',
    'votes' : 'Votes',
    'director' : 'Director',
    'writer' : 'Writer',
    'star' : 'Star',
    'country' : 'Country',
    'budget' : 'Budget',
    'gross' : ' Gross Revenue',
    'company' : 'Company',
    'runtime' : 'Runtime'
}

df = df.rename(columns=column_mapping)

In [4]:
# Check for empty spaces
isnas = df.isna()
print(isnas)

       Name  Rating  Genre   Year  Released  Score  Votes  Director  Writer  \
0     False   False  False  False     False  False  False     False   False   
1     False   False  False  False     False  False  False     False   False   
2     False   False  False  False     False  False  False     False   False   
3     False   False  False  False     False  False  False     False   False   
4     False   False  False  False     False  False  False     False   False   
...     ...     ...    ...    ...       ...    ...    ...       ...     ...   
7663  False    True  False  False     False  False  False     False   False   
7664  False    True  False  False     False  False  False     False   False   
7665  False    True  False  False     False  False  False     False   False   
7666  False    True  False  False     False   True   True     False   False   
7667  False    True  False  False     False  False  False     False   False   

       Star  Country  Budget   Gross Revenue  Compa

In [5]:
# Remove rows with null values
df = df.dropna()
isnas2 = df.isna()
print(isnas2)

       Name  Rating  Genre   Year  Released  Score  Votes  Director  Writer  \
0     False   False  False  False     False  False  False     False   False   
1     False   False  False  False     False  False  False     False   False   
2     False   False  False  False     False  False  False     False   False   
3     False   False  False  False     False  False  False     False   False   
4     False   False  False  False     False  False  False     False   False   
...     ...     ...    ...    ...       ...    ...    ...       ...     ...   
7648  False   False  False  False     False  False  False     False   False   
7649  False   False  False  False     False  False  False     False   False   
7650  False   False  False  False     False  False  False     False   False   
7651  False   False  False  False     False  False  False     False   False   
7652  False   False  False  False     False  False  False     False   False   

       Star  Country  Budget   Gross Revenue  Compa

In [6]:
df['Votes'] = df['Votes'].astype('int')
df['Budget'] = df['Budget'].astype('int')
df['Runtime'] = df['Runtime'].astype('int')
df['Score'] = df['Score'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
# Note: For some reason the 'Gross Revenue' column is not detected making me unable to remove trailing 0s in the column
#df['Gross Revenue'] = df['Gross Revenue'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
df

,Name,Rating,Genre,Year,Released,Score,Votes,Director,Writer,Star,Country,Budget,Gross Revenue,Company,Runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.000000,Warner Bros.,146
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.000000,Columbia Pictures,104
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.000000,Lucasfilm,124
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.000000,Paramount Pictures,88
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.000000,Orion Pictures,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,Bad Boys for Life,R,Action,2020,"January 17, 2020 (United States)",6.6,140000,Adil El Arbi,Peter Craig,Will Smith,United States,90000000,426505244.000000,Columbia Pictures,124
7649,Sonic the Hedgehog,PG,Action,2020,"February 14, 2020 (United States)",6.5,102000,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000,319715683.000000,Paramount Pictures,99
7650,Dolittle,PG,Adventure,2020,"January 17, 2020 (United States)",5.6,53000,Stephen Gaghan,Stephen Gaghan,Robert Downey Jr.,United States,175000000,245487753.000000,Universal Pictures,101
7651,The Call of the Wild,PG,Adventure,2020,"February 21, 2020 (United States)",6.8,42000,Chris Sanders,Michael Green,Harrison Ford,Canada,135000000,111105497.000000,20th Century Studios,100


# Data Cleaning (UCI Dataset)

In [8]:
# NOTE: UCI dataset not complete. The html file is formatted in way that makes creating a dataframe from it 
# difficult to accomplish. 

# url = 'Datasets/main.html'

# MAIN_df = pd.read_html(url)

# total_dataframe = []

# # Remove any NaNs
# for i in range(len(MAIN_df)):
#     MAIN_df[i].dropna(inplace=True)
# #    print(pd.isna(MAIN_df[i]))

# # Create a list of dataframes created from read_html(url)
# for j, df in enumerate(MAIN_df):
#     total_dataframe.append(MAIN_df[j])
    
# combined = pd.concat(total_dataframe, ignore_index=True)
  
# combined


In [464]:
# Function to pop indices that don't have exactly 12 columns
def pop_multiple_indices(dataframes, indices):
    # Sort the indices in descending order so that popping does not affect the subsequent indices
    indices.sort(reverse=True)
    
    # Pop DataFrames from the list at the specified indices
    popped_dfs = [dataframes.pop(index) for index in indices]
    
    return popped_dfs

In [1016]:
url = 'Datasets/main.html'

MAIN_df = pd.read_html(url)

total_dataframe = []


# Renaming columns
column_mapping3 = {
    'title' : 'Title',
    'prds' : 'Producer',
    'prc.1' : 'Genre',
    'cat' : 'Awards',
    'ct' : "Genre",
    'aw' : "Awards",
    'st' : 'Studio',
    'st.' : 'Studio',
    'producers' : 'Producer',
    'prodrs' : 'Producer',
    'prods' : 'Producer',
    'prds' : 'Producer'
}
for i in range(len(MAIN_df)):
    MAIN_df[i] = MAIN_df[i].rename(columns=column_mapping3)

In [1017]:
# Check number of columns in each dataframe
counter = 0
indices = []
for i in range(len(MAIN_df)):
    num_columns = MAIN_df[i].shape[1]
    #print("Number of columns at index " + str(i) + ":", num_columns)
    if ((num_columns < 12) or (num_columns == 13)):
        #print("Pop at index " + str(i))
        counter = counter + 1
        indices.append(i)

# Check how many dataframes do not have 12 columns
# print(counter)
# print(indices)


In [1018]:
# Assuming dfs is your list of dataframes and indices is a list of indices you want to pop
popped_dfs = pop_multiple_indices(MAIN_df, indices)

counter2 = 0
indices2 = []
for i in range(len(MAIN_df)):
    num_columns = MAIN_df[i].shape[1]
    #print("Number of columns at index " + str(i) + ":", num_columns)
    if ((num_columns < 12) or (num_columns == 13)):
        #print("Pop at index " + str(i))
        counter2 = counter2 + 1
        indices2.append(i)

# Check that all dataframes have 12 columns
print("Number of dataframes that don't have 12 columns: " + str(counter2))
print("Indices where dataframe does not have 12 columns:", indices2)

Number of dataframes that don't have 12 columns: 0
Indices where dataframe does not have 12 columns: []


In [1019]:
# Drop useless columns
for i in range(len(MAIN_df)):
    # Drop unneeded columns
    MAIN_df[i] = MAIN_df[i].drop(MAIN_df[i].columns[[0, 6, 9, 10, 11]], axis=1)
    #print('Dataframe ' + str(i) + ' is good')

# print(MAIN_df[0].columns)
# print(MAIN_df[567].columns)


In [1020]:
# Remove NaNs
clean_dfs = []
for i in range(len(MAIN_df)):
    clean_df = MAIN_df[i].dropna()  # Drop rows containing NaN values
    clean_dfs.append(clean_df)

MAIN_df = clean_dfs

In [1021]:
# Checking for out of place column names
counter3 = 0
indices3 = []
for i in range(len(MAIN_df)):
    if ((MAIN_df[i].columns[0] == 1) or (MAIN_df[i].columns[0] == 'Unnamed: 1')):
        counter3 = counter3 + 1
        indices3.append(i)

print(counter3)
print(indices3)

# Removing out of place column names
popped_dfs2 = pop_multiple_indices(MAIN_df, indices3)


43
[506, 651, 773, 953, 1007, 1219, 1790, 1791, 1876, 1909, 1997, 2022, 2111, 2538, 2588, 2608, 2611, 2623, 2632, 2675, 2687, 2754, 2757, 2766, 2776, 2818, 2819, 2822, 2823, 2826, 2827, 2828, 2830, 2831, 2836, 2837, 2838, 2840, 2841, 2845, 2847, 2849, 2850]


In [1022]:
# Eliminate unneccessary characters from 'Producer' (WIP)
# Current problem: "AttributeError: Can only use .str accessor with string values!"
# for i in range(len(MAIN_df)):
# # for i in range(1000):
#     contains_character1 = MAIN_df[i]['Producer'].str.contains('P:')
#     contains_character2 = MAIN_df[i]['Producer'].str.contains('PN:')
#     check1 = contains_character1.any()
#     check2 = contains_character2.any()
#     if (check1 == True):
#         MAIN_df[i]['Producer'] = MAIN_df[i]['Producer'].str.replace('P:', '', regex=True)
#     elif (check2 == True):
#         MAIN_df[i]['Producer'] = MAIN_df[i]['Producer'].str.replace('PN:', '', regex=True)
#     print("index " + str(i) + " is good")

In [1028]:
MAIN_df[0]

,Title,@1940,D:Hitchcock,Producer,Studio,Genre,Awards
0,T:Rebecca,1940,D:Hitchcock,P:Selznick,"St:Selznick, SD:U.A.",Dram,"AA, AAN dir, H****"
1,T:Foreign Correspondent,1940,D:Hitchcock,P:Wanger,St:U.A.,Susp,"H****, AAN"
4,T:Saboteur,1942,D:Hitchcock,"P:F.Lloyd, Skirball",St:Universal,Susp,H***
6,T:Lifeboat,1943,D:Hitchcock,P:MacGowan,St:Fox,Susp,"H**, AAN dir"
7,T:Spellbound,1945,D:Hitchcock,P:Selznick,"St:Selznick, SD:U.A.",Susp,"H**, AAN, AAN dir"
8,T:Notorious,1946,D:Hitchcock,P:Hitchcock,St:RKO,Susp,H***
9,T:The Paradine Case,1947,D:Hitchcock,P:Selznick,"St:Selznick, SD:U.A.",Susp,H**
10,T:Rope,1948,D:Hitchcock,"P:Bernstein, Hitchcock","St:Transatlantic, SD:Warners",Susp,H**
13,T:Strangers on a Train,1951,D:Hitchcock,P:Hitchcock,St:Warners,Susp,H***
15,T:Dial M for Murder,1954,D:Hitchcock,P:Hitchcock,St:Warners,Susp,H**


# Data Cleaning (BoxOfficeCollections.csv)

In [1024]:
BoxOff_df = pd.read_csv('Datasets/BoxOfficeCollections.csv')

In [1025]:
print(list(BoxOff_df.columns.values))

# Renaming columns
column_mapping2 = {
    'Imdb_genre' : 'IMDB Genre',
    'metascore' : 'Metascore',
    'time_minute' : 'Time(Min)' 
}

BoxOff_df = BoxOff_df.rename(columns=column_mapping2)
BoxOff_df.head()

['Movie', 'Year', 'Score', 'Adjusted Score', 'Director', 'Cast', 'Consensus', 'Box Office Collection', 'Imdb_genre', 'IMDB Rating', 'metascore', 'time_minute', 'Votes']


,Movie,Year,Score,Adjusted Score,Director,Cast,Consensus,Box Office Collection,IMDB Genre,IMDB Rating,Metascore,Time(Min),Votes
0,Hot Rod,2007,39,42.918000,Akiva Schaffer,"Andy Samberg, Jorma Taccone, Bill Hader, Danny...","For Rod Kimball (Andy Samberg), performing stu...",14371564.000000,Comedy,6.700000,43.000000,88.000000,84956.000000
1,Game Night,2018,85,99.838000,John Francis Daley,"Jason Bateman, Rachel McAdams, Kyle Chandler, ...",Max and Annie's weekly game night gets kicked ...,117378084.000000,Comedy,6.900000,66.000000,100.000000,229292.000000
2,The First Wives Club,1996,49,53.174000,Hugh Wilson,"Goldie Hawn, Bette Midler, Diane Keaton, Maggi...",Despondent over the marriage of her ex-husband...,181489203.000000,Comedy,6.400000,58.000000,103.000000,48413.000000
3,Scary Movie,2000,52,54.973000,Keenen Ivory Wayans,"Shawn Wayans, Marlon Wayans, Cheri Oteri, Shan...","Defying the very notion of good taste, Scary M...",277200000.000000,Comedy,6.200000,48.000000,88.000000,254927.000000
4,Blockers,2018,84,96.883000,Kay Cannon,"Leslie Mann, Ike Barinholtz, John Cena, Kathry...","Julie, Kayla and Sam are three high school sen...",94523781.000000,Comedy,6.200000,69.000000,102.000000,78498.000000


In [1026]:

# Remove trailing zeros from the 'GrossRevenue' column
BoxOff_df['Adjusted Score'] = BoxOff_df['Adjusted Score'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
BoxOff_df['Box Office Collection'] = BoxOff_df['Box Office Collection'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
BoxOff_df['IMDB Rating'] = BoxOff_df['IMDB Rating'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
BoxOff_df['Metascore'] = BoxOff_df['Metascore'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
BoxOff_df['Time(Min)'] = BoxOff_df['Time(Min)'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)
BoxOff_df['Votes'] = BoxOff_df['Votes'].apply(lambda x: str(x).rstrip('0').rstrip('.') if isinstance(x, float) else x)

In [1027]:
# Remove rows with null values
BoxOff_df = BoxOff_df.dropna()
isnas3 = BoxOff_df.isna()
#print(isnas3)

BoxOff_df

,Movie,Year,Score,Adjusted Score,Director,Cast,Consensus,Box Office Collection,IMDB Genre,IMDB Rating,Metascore,Time(Min),Votes
0,Hot Rod,2007,39,42.918,Akiva Schaffer,"Andy Samberg, Jorma Taccone, Bill Hader, Danny...","For Rod Kimball (Andy Samberg), performing stu...",14371564,Comedy,6.7,43,88,84956
1,Game Night,2018,85,99.838,John Francis Daley,"Jason Bateman, Rachel McAdams, Kyle Chandler, ...",Max and Annie's weekly game night gets kicked ...,117378084,Comedy,6.9,66,100,229292
2,The First Wives Club,1996,49,53.174,Hugh Wilson,"Goldie Hawn, Bette Midler, Diane Keaton, Maggi...",Despondent over the marriage of her ex-husband...,181489203,Comedy,6.4,58,103,48413
3,Scary Movie,2000,52,54.973,Keenen Ivory Wayans,"Shawn Wayans, Marlon Wayans, Cheri Oteri, Shan...","Defying the very notion of good taste, Scary M...",277200000,Comedy,6.2,48,88,254927
4,Blockers,2018,84,96.883,Kay Cannon,"Leslie Mann, Ike Barinholtz, John Cena, Kathry...","Julie, Kayla and Sam are three high school sen...",94523781,Comedy,6.2,69,102,78498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,Stalker,1979,100,103.012,Andrei Tarkovsky,"Aleksandr Kajdanovsky, Nikolay Grinko, Anatoli...","In an unnamed country at an unspecified time, ...",296334,Drama,8.1,nan,162,131448
1363,Tampopo,1985,100,105.082,Juzo Itami,"Tsutomu Yamazaki, Nobuko Miyamoto, Ken Watanab...",Two Japanese milk-truck drivers (Tsutomu Yamaz...,nan,Comedy,7.9,87,114,18557
1367,Three Colors: Red,1994,100,101.27,Krzysztof Kieslowski,"Irène Jacob, Jean-Louis Trintignant, Frédériqu...",Part-time model Valentine (Irène Jacob) meets ...,nan,Drama,8.1,100,99,101519
1369,Top Hat,1935,100,107.903,Mark Sandrich,"Fred Astaire, Ginger Rogers, Edward Everett Ho...",The story centers on wealthy Dale Tremont (Gin...,3202000,Comedy,7.7,92,101,19036
